In [ ]:
import numpy as np, matplotlib.pyplot as plt
from pyhnc import *

Set up basic variables for Ornstein-Zernike solver.

In [ ]:
N = 2**14
Δr = 0.01
grid = Grid(N, Δr)
r, q = grid.r, grid.q

verbose = True

alpha = 0.5
niters = 1000
tol = 1e-12
solvent = Solver(grid, alpha=alpha, niters=niters, tol=tol)

First test: solve for various mixtures of DPD particles (mono, binary, ternary).

In [ ]:
A0, rcut = 25, 1
v = potentials.DPD(A0, rcut)
ρ = 3.0

sol1 = solvent.solve(v, ρ, monitor=verbose)

for m in [2, 3]:
    M = np.ones((m,m))
    v = potentials.DPD(A0 * M, rcut * M)
    xi = np.ones(m) / m
    ρi = xi * ρ

    sol2 = solvent.solve(v, ρi, monitor=verbose)

    for idx in np.ndindex(sol2.h.shape[:-1]):
        assert np.allclose(sol2.h[idx], sol1.h)

Now test that we can solve with different interaction coefficients. The cross-species distribution functions should now differ from the same-species correlations:

In [ ]:
A0, rcut = 25, 1
Across = A0 + 5
M = np.ones((2, 2))
Aij = np.array([[A0, Across], [Across, A0]])
v1 = potentials.DPD(A0, rcut)
v2 = potentials.DPD(Aij, rcut)

(v00, v01), (v10, v11) = v2.potential(r)
assert np.allclose(v00, v1.potential(r))
# Should be identical from index symmetry.
assert np.allclose(v01, v10)
# Should be identical by physical symmetry.
assert np.allclose(v00, v11)

pl1, = plt.plot(r, v00, label=r'$v_{00}$')
pl2, = plt.plot(r, v01, '-.', label=r'$v_{01}$')
plt.xlim([0, 5])
plt.xlabel('$r$')
plt.ylabel(r'$v_{ij}(r)$')
plt.legend(loc='best')

ρ = 3.0
sol1 = solvent.solve(v1, ρ, monitor=verbose)
sol2 = solvent.solve(v2, 0.5*ρ * np.ones(2), monitor=verbose)
assert solvent.converged

(g00, g01), (g10, g11) = sol2.g
# Should be identical from index symmetry.
assert np.allclose(g01, g10)
# Should be identical by physical symmetry.
assert np.allclose(g00, g11)

plt.figure()
pl, = plt.plot(r, sol1.g, ':', label='mono')
plt.plot(r, g00, '-', c=pl.get_color(), label=r'$g_{00}(r)$')
plt.plot(r, g01, '-.', label=r'$g_{01}(r)$')
plt.legend(loc='best')
plt.xlim([0, 5])
plt.xlabel('$r$')
plt.ylabel(r'$g_{ij}(r)$')
plt.show()

Now check we can solve for a binary mixture of soft charged beads. 

In [ ]:
sigma = 0.5
alpha = 1/(2*sigma**2)
v = potentials.GaussianIon([1, -1], alpha)
print(v)

ρ = 3
xi = np.ones(v.nspecies) / v.nspecies
ρi = xi * ρ

solvent.solve(v, ρi, monitor=verbose, restart=True)
g00, g01 = solvent.g[0]
v00, v01 = v.potential(r)[0]

pl1, = plt.plot(r, g00, label=r'$g_{++}$')
pl2, = plt.plot(r, g01, label=r'$g_{+-}$')
# plt.plot(r, np.exp(-v00), '--', c=pl1.get_color())
# plt.plot(r, np.exp(-v01), '--', c=pl2.get_color())

plt.xlim([0, 5])
plt.xlabel('$r$')
plt.ylabel('$g(r)$')
plt.legend(loc='best')
plt.show()

Solve a ternary DPD system with one neutral bead and two charged ions. We take parameters consistent with [Warren et al. JCP 138 (2013)](http://dx.doi.org/10.1063/1.4807057) so we can compare with published Monte-Carlo data (the points below):

In [ ]:
class DPDGaussianIon(potentials.Potential):
    r"""DPD bead with an additional Gaussian-distributed soft electrostatic
    potential. Cf. documentation for `potentials.DPD` and
    `potentials.GaussianIon` for details on these components.
    """

    def __getstate__(self):
        return {'dpd': self.dpd.copy(),
                'ion': self.ion.copy()}

    def __repr__(self):
        return rf'<DPDGaussianIon dpd={self.dpd} ion={self.ion}>'

    def __init__(self, A: float | NDArray,
                 z: float | NDArray, α: float,
                 rcut: float | NDArray=1.,
                 lB: float=1.):

        self.dpd = potentials.DPD(A, rcut)
        self.ion = potentials.GaussianIon(z, α, lB)
        assert self.dpd.nspecies == self.ion.nspecies
        self.long = self.ion.long
        self.short = potentials.ShortRangeResidual(self, self.long)

    @property
    def nspecies(self):
        assert self.dpd.nspecies == self.ion.nspecies
        return self.dpd.nspecies

    def potential(self, r: float | NDArray):
        return self.dpd.potential(r) + self.ion.potential(r)

    def force(self, r: float | NDArray):
        return self.dpd.force(r) + self.ion.force(r)


A0 = 25
Aij = A0 * np.ones((3, 3))
rcut = 1
sigma = 0.5
alpha = 1/(2*sigma**2)
v = DPDGaussianIon(Aij, [0, 1, -1], alpha, rcut)
print(v)

ρ = 3
ρz = 0.1
ρ0 = ρ - ρz
ρi = np.array([ρ0, 0.5*ρz, 0.5*ρz])
assert np.isclose(np.sum(ρi), ρ)

(v00, v01, v02), (v10, v11, v12), (v20, v21, v22) = v.potential(r)
# Should be identical from index symmetry.
assert np.allclose(v01, v10)
assert np.allclose(v02, v20)
assert np.allclose(v12, v21)
# Should be identical by physical symmetry.
assert np.allclose(v01, v02)
assert np.allclose(v11, v22)

# pl1, = plt.plot(r, v00, label=r'$v_{00}$')
# pl2, = plt.plot(r, v01, '-.', label=r'$v_{0+}$')
# pl3, = plt.plot(r, v11, '--', label=r'$v_{++}$')
# pl4, = plt.plot(r, v12, ':', label=r'$v_{+-}$')
# plt.xlim([0, 5])
# plt.xlabel('$r$')
# plt.ylabel(r'$v_{ij}(r)$')
# plt.legend(loc='best')
# plt.show()

solvent.solve(v, ρi, monitor=verbose, restart=True)
(g00, g01, g02), (g10, g11, g12), (g20, g21, g22) = solvent.g
# Should be identical from index symmetry.
assert np.allclose(g01, g10)
assert np.allclose(g02, g20)
assert np.allclose(g12, g21)
# Same by charge symmetry.
assert np.allclose(g01, g02)
assert np.allclose(g11, g22)

pl1, = plt.plot(r, g00, label=r'$g_{00}$')
# pl2, = plt.plot(r, g01, '-.', label=r'$g_{0+}$')
pl3, = plt.plot(r, g11, '-', label=r'$g_{++}$')
pl4, = plt.plot(r, g12, '--', label=r'$g_{+-}$')

# plt.plot(r, np.exp(-v00), ':', c=pl1.get_color())
# plt.plot(r, np.exp(-v01), ':', c=pl2.get_color())
# plt.plot(r, np.exp(-v11), ':', c=pl3.get_color())
# plt.plot(r, np.exp(-v12), ':', c=pl4.get_color())

rref, gref, gerr = np.genfromtxt('data/warren2013_g00.txt').T
plt.errorbar(rref, gref, gerr, ls='None', capsize=1, c=pl1.get_color())
rref, gref, gerr = np.genfromtxt('data/warren2013_g++.txt').T
plt.errorbar(rref, gref, gerr, ls='None', capsize=1, c=pl3.get_color())
rref, gref, gerr = np.genfromtxt('data/warren2013_g+-.txt').T
plt.errorbar(rref, gref, gerr, ls='None', capsize=1, c=pl4.get_color())

plt.xlim([0, 5])
plt.ylim([0, 2])
plt.yticks(np.arange(0, 2.1, 0.5))
plt.xlabel('$r$')
plt.ylabel(r'$g_{ij}(r)$')
plt.legend(loc='best')
plt.show()